In [40]:
#  Market News 2 NAPI code to run locally

import os
import logging
from typing import Optional, Tuple, Dict, Any
from datetime import datetime

import pandas as pd
from google.cloud import bigquery
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

class NewsDataProcessorError(Exception):
    """Custom exception for NewsDataProcessor errors."""
    pass

class NewsDataProcessor:
    def __init__(self, project_id: str, dataset_id: str, logger: Optional[logging.Logger] = None):
        # Validate inputs and set up logging
        self._validate_input_parameters(project_id, dataset_id)
        self.logger = logger or self._setup_logger()

        # Initialize BigQuery client
        try:
            self.client = bigquery.Client(project=project_id)
        except Exception as e:
            self.logger.error(f"Failed to initialize BigQuery client: {e}")
            raise NewsDataProcessorError(f"BigQuery client initialization failed: {e}")

        # Store project and dataset IDs
        self.project_id = project_id
        self.dataset_id = dataset_id

        # Initialize VADER sentiment analyzer
        self.vader_analyzer = SentimentIntensityAnalyzer()

    def _validate_input_parameters(self, project_id: str, dataset_id: str):
        """Ensure the provided project and dataset IDs are valid strings."""
        if not project_id or not isinstance(project_id, str):
            raise NewsDataProcessorError("Invalid project_id. Must be a non-empty string.")
        if not dataset_id or not isinstance(dataset_id, str):
            raise NewsDataProcessorError("Invalid dataset_id. Must be a non-empty string.")

    def _setup_logger(self) -> logging.Logger:
        """Set up a logger for debugging and tracking operations."""
        logger = logging.getLogger(self.__class__.__name__)
        logger.setLevel(logging.DEBUG)  # Set to DEBUG for detailed logs
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.DEBUG)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)
        return logger

    def calculate_vader_sentiment(self, text: Optional[str]) -> float:
        """Calculate sentiment using VADER sentiment analyzer."""
        if not text or not isinstance(text, str):
            return 0.0
        try:
            sentiment = self.vader_analyzer.polarity_scores(text)
            return sentiment.get("compound", 0.0)  # Return compound sentiment score
        except Exception as e:
            self.logger.warning(f"VADER sentiment analysis failed: {e}")
            return 0.0

    def ensure_table_exists(self, table_id: str):
        """Ensure the target BigQuery table exists with the required schema."""
        table_ref = f"{self.project_id}.{self.dataset_id}.{table_id}"
        try:
            self.client.get_table(table_ref)
            self.logger.info(f"Table {table_ref} already exists.")
        except Exception:
            self.logger.info(f"Table {table_ref} does not exist. Creating it...")
            schema = [
                bigquery.SchemaField("ticker", "STRING"),
                bigquery.SchemaField("title", "STRING"),
                bigquery.SchemaField("summary", "STRING"),
                bigquery.SchemaField("publisher", "STRING"),
                bigquery.SchemaField("link", "STRING"),
                bigquery.SchemaField("publish_date", "DATETIME"),
                bigquery.SchemaField("type", "STRING"),
                bigquery.SchemaField("related_tickers", "STRING"),
                bigquery.SchemaField("source", "STRING"),
                bigquery.SchemaField("lexical_diversity", "FLOAT"),
                bigquery.SchemaField("reliability_score", "FLOAT"),
                bigquery.SchemaField("textblob_sentiment", "FLOAT"),
                bigquery.SchemaField("vader_sentiment", "FLOAT"),
                bigquery.SchemaField("bert_sentiment", "FLOAT"),
                bigquery.SchemaField("bert_confidence", "FLOAT"),
                bigquery.SchemaField("word_count", "INTEGER"),
                bigquery.SchemaField("headline_sentiment", "FLOAT"),
            ]
            table = bigquery.Table(table_ref, schema=schema)
            try:
                self.client.create_table(table)
                self.logger.info(f"Table {table_ref} created successfully.")
            except Exception as e:
                self.logger.error(f"Failed to create table {table_ref}: {e}")
                raise NewsDataProcessorError(f"Table creation failed: {e}")

    def process_and_replace_data(self, source_table_id: str, target_table_id: str, batch_size: int = 1000) -> Dict[str, Any]:
        """Process data from the source table and replace data in the target table."""
        source_table = f"{self.project_id}.{self.dataset_id}.{source_table_id}"
        target_table = f"{self.project_id}.{self.dataset_id}.{target_table_id}"

        try:
            # Query data from the source table
            query = f"""
            SELECT 
                ticker, title, summary, publisher, link, publish_date, type, 
                related_tickers, source, lexical_diversity, reliability_score, summary_sentiment
            FROM `{source_table}`
            """

            new_data = self.client.query(query).to_dataframe()

            if new_data.empty:
                return {"status": "success", "message": "No new data to process", "rows_processed": 0}

            # Perform transformations and calculations
            new_data["word_count"] = new_data["summary"].apply(lambda x: len(str(x).split()))  # Calculate word count
            new_data["headline_sentiment"] = new_data["title"].apply(self.calculate_vader_sentiment)  # Analyze title sentiment
            new_data["vader_sentiment"] = new_data["summary"].apply(self.calculate_vader_sentiment)  # Analyze summary sentiment

            # Placeholder for BERT sentiment
            new_data["bert_sentiment"] = 0.0
            new_data["bert_confidence"] = 0.0

            # Replace the target table with the new data
            job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)
            self.client.load_table_from_dataframe(new_data, target_table, job_config=job_config).result()

            success_msg = f"Data successfully replaced in {target_table}. Rows added: {len(new_data)}"
            self.logger.info(success_msg)

            return {"status": "success", "message": success_msg, "rows_processed": len(new_data)}

        except Exception as e:
            error_msg = f"Error processing data: {e}"
            self.logger.error(error_msg)
            return {"status": "error", "message": error_msg, "rows_processed": 0}

if __name__ == "__main__":
    # Configuration
    project_id = os.getenv("GCP_PROJECT_ID", "trendsense")
    dataset_id = os.getenv("BQ_DATASET_ID", "market_data")
    source_table_id = os.getenv("SOURCE_TABLE_ID", "Market_News_NAPI_Temp")
    target_table_id = os.getenv("TARGET_TABLE_ID", "Market_News_NAPI")

    # Initialize the processor and logger
    processor = NewsDataProcessor(project_id, dataset_id)
    processor.ensure_table_exists(target_table_id)

    # Process and replace data
    result = processor.process_and_replace_data(source_table_id, target_table_id)
    print(result)

















2025-01-24 11:19:56,551 - NewsDataProcessor - INFO - Table trendsense.market_data.Market_News_NAPI already exists.
2025-01-24 11:19:56,551 - NewsDataProcessor - INFO - Table trendsense.market_data.Market_News_NAPI already exists.
c:\Users\BryceDaniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-01-24 11:20:12,244 - NewsDataProcessor - INFO - Data successfully replaced in trendsense.market_data.Market_News_NAPI. Rows added: 19575
2025-01-24 11:20:12,244 - NewsDataProcessor - INFO - Data successfully replaced in trendsense.market_data.Market_News_NAPI. Rows added: 19575


{'status': 'success', 'message': 'Data successfully replaced in trendsense.market_data.Market_News_NAPI. Rows added: 19575', 'rows_processed': 19575}


In [38]:
!pip install transformers














   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------------------ --------------------- 4.5/9.7 MB 30.0 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 40.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 68.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Yahoo Extract 1/24.  This code has no date restriction

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob
from newspaper import Article
import nltk
import os

import nltk
nltk.download('punkt')


def fetch_article_summary(link):
    try:
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"[ERROR] Failed to fetch or summarize article: {e}")
        return "No summary available."

def calculate_sentiment(text):
    try:
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    except Exception as e:
        print(f"[ERROR] Sentiment analysis failed: {e}")
        return 0

def label_sentiment(score):
    if score > 0.35:
        return "Bullish"
    elif 0.15 < score <= 0.35:
        return "Somewhat-Bullish"
    elif -0.15 <= score <= 0.15:
        return "Neutral"
    elif -0.35 <= score < -0.15:
        return "Somewhat-Bearish"
    else:
        return "Bearish"

def get_market_news(tickers):
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    if True:
                    #if publish_date >= one_day_ago and item.get('type', '').lower() == 'story':
                        link = item.get('link', '')
                        summary = fetch_article_summary(link) if link else "No summary available."
                        sentiment_score = calculate_sentiment(summary)
                        sentiment_label = label_sentiment(sentiment_score)

                        news_item = {
                            'ticker': ticker,
                            'title': item.get('title', ''),
                            'summary': summary,
                            'publisher': item.get('publisher', ''),
                            'link': link,
                            'publish_date': datetime.fromtimestamp(publish_timestamp),
                            'type': item.get('type', ''),
                            'related_tickers': ', '.join(item.get('relatedTickers', [])),
                            'source': 'yahoo',
                            'overall_sentiment_score': sentiment_score,
                            'overall_sentiment_label': sentiment_label,
                        }
                        all_news.append(news_item)
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")
        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")
    return pd.DataFrame(all_news)

def save_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False)
        print(f"[INFO] Data successfully saved to {filename}")
    except Exception as e:
        print(f"[ERROR] Failed to save data to CSV: {e}")

def fetch_and_save_market_news():
    indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
    market_news = get_market_news(tickers=indices)
    if not market_news.empty:
        market_news['category'] = 'General'

    tech_stocks = [
        'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
        'BWXT', 'ARBK', 'AMD', 'NVDA', 'BTC', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
        'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META','RGTI','QUBT',
        'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
    ]
    tech_news = get_market_news(tickers=tech_stocks)
    if not tech_news.empty:
        tech_news['category'] = 'Tech'

    combined_news = pd.concat([market_news, tech_news], ignore_index=True)

    if not combined_news.empty:
        save_to_csv(combined_news, "market_news.csv")
    else:
        print("[INFO] No news data to save.")

if __name__ == "__main__":
    fetch_and_save_market_news()










In [41]:
!pip install nltk


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
# Yahoo Extract with date restriction 

import nltk
import os

# Explicitly set the nltk_data path
nltk_data_path = r"C:\Users\BryceDaniel\OneDrive - Lincoln Telephone Company\MSBA\GitHub\TrendSense\Market News\Market_News_Yahoo_Extract_Function\nltk_data"
nltk.data.path.append(nltk_data_path)

# Ensure 'punkt' is downloaded into the correct folder
nltk.download('punkt', download_dir=nltk_data_path)

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob


def calculate_sentiment(text):
    try:
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    except Exception as e:
        print(f"[ERROR] Sentiment analysis failed: {e}")
        return 0

def label_sentiment(score):
    if score > 0.35:
        return "Bullish"
    elif 0.15 < score <= 0.35:
        return "Somewhat-Bullish"
    elif -0.15 <= score <= 0.15:
        return "Neutral"
    elif -0.35 <= score < -0.15:
        return "Somewhat-Bearish"
    else:
        return "Bearish"

def get_market_news(tickers, days_back=2):
    all_news = []
    today = datetime.now().date()
    cutoff_date = today - timedelta(days=days_back)

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try:
            news = stock.news
            for item in news:
                try:
                    # Extract publish timestamp and date
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Only process news within the desired date range
                    if publish_date >= cutoff_date:
                        title = item.get('title', '')
                        sentiment_score = calculate_sentiment(title)
                        sentiment_label = label_sentiment(sentiment_score)

                        news_item = {
                            'ticker': ticker,
                            'title': title,
                            'summary': title,  # Replicate title in the summary column
                            'publisher': item.get('publisher', ''),
                            'link': item.get('link', ''),
                            'publish_date': datetime.fromtimestamp(publish_timestamp),
                            'type': item.get('type', ''),
                            'related_tickers': ', '.join(item.get('relatedTickers', [])),
                            'source': 'yahoo',
                            'overall_sentiment_score': sentiment_score,
                            'overall_sentiment_label': sentiment_label,
                        }
                        all_news.append(news_item)
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")
        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")
    return pd.DataFrame(all_news)

def save_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False)
        print(f"[INFO] Data successfully saved to {filename}")
    except Exception as e:
        print(f"[ERROR] Failed to save data to CSV: {e}")

def fetch_and_save_market_news():
    indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
    market_news = get_market_news(tickers=indices)
    if not market_news.empty:
        market_news['category'] = 'General'

    tech_stocks = [
        'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
        'BWXT', 'ARBK', 'AMD', 'NVDA', 'BTC', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
        'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META','RGTI','QUBT',
        'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
    ]
    tech_news = get_market_news(tickers=tech_stocks)
    if not tech_news.empty:
        tech_news['category'] = 'Tech'

    combined_news = pd.concat([market_news, tech_news], ignore_index=True)

    if not combined_news.empty:
        save_to_csv(combined_news, "market_news.csv")
    else:
        print("[INFO] No news data to save.")

if __name__ == "__main__":
    fetch_and_save_market_news()


[nltk_data] Downloading package punkt to C:\Users\BryceDaniel\OneDrive
[nltk_data]     - Lincoln Telephone
[nltk_data]     Company\MSBA\GitHub\TrendSense\Market
[nltk_data]     News\Market_News_Yahoo_Extract_Function\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


[INFO] Data successfully saved to market_news.csv


In [34]:
import nltk
import os

# Set the custom path for nltk_data
nltk_data_path = r"C:\Users\BryceDaniel\OneDrive - Lincoln Telephone Company\MSBA\GitHub\TrendSense\Market News\Market_News_Yahoo_Extract_Function\nltk_data"
nltk.data.path.append(nltk_data_path)

# Force redownload of punkt
nltk.download('punkt', download_dir=nltk_data_path)




[nltk_data] Downloading package punkt to C:\Users\BryceDaniel\OneDrive
[nltk_data]     - Lincoln Telephone
[nltk_data]     Company\MSBA\GitHub\TrendSense\Market
[nltk_data]     News\Market_News_Yahoo_Extract_Function\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
import requests
import yfinance as yf

class StockPriceTargetRetriever:
    def __init__(self, api_key=None):
        """
        Initialize the Stock Price Target Retriever
        
        :param api_key: API key for paid services (optional)
        """
        self.api_key = api_key
    
    def get_yahoo_finance_target(self, symbol):
        """
        Retrieve price targets and recommendations from Yahoo Finance
        
        :param symbol: Stock ticker symbol
        :return: Dictionary with recommendations and price targets
        """
        try:
            # Fetch the stock information
            stock = yf.Ticker(symbol)
            
            # Fetch analyst recommendations
            recommendations = stock.recommendations
            
            # Fetch analyst price targets
            info = stock.info
            
            # Extract price target information from stock info
            price_targets = {
                'current_price': info.get('currentPrice'),
                'target_high_price': info.get('targetHighPrice'),
                'target_low_price': info.get('targetLowPrice'),
                'target_mean_price': info.get('targetMeanPrice'),
                'target_median_price': info.get('targetMedianPrice')
            }
            
            return {
                'recommendations': recommendations,
                'price_targets': price_targets
            }
        except Exception as e:
            print(f"Error fetching Yahoo Finance data: {e}")
            return None
    
    def get_alpha_vantage_overview(self, symbol):
        """
        Retrieve stock overview from Alpha Vantage
        
        :param symbol: Stock ticker symbol
        :return: Dictionary of stock overview data
        """
        if not self.api_key:
            raise ValueError("Alpha Vantage requires an API key")
        
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={self.api_key}'
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from Alpha Vantage: {e}")
            return None
    
    def get_financial_modeling_prep_target(self, symbol):
        """
        Retrieve price targets from Financial Modeling Prep
        
        :param symbol: Stock ticker symbol
        :return: List of price target data
        """
        if not self.api_key:
            raise ValueError("Financial Modeling Prep requires an API key")
        
        url = f'https://financialmodelingprep.com/api/v3/price-target?symbol={symbol}&apikey={self.api_key}'
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from Financial Modeling Prep: {e}")
            return None

def main():
    # Initialize the retriever
    retriever = StockPriceTargetRetriever()
    
    # Retrieve price targets for Apple (AAPL)
    symbol = 'ASTS'
    
    # Yahoo Finance (completely free)
    yahoo_targets = retriever.get_yahoo_finance_target(symbol)
    
    # Print results with error handling
    if yahoo_targets:
        print("Yahoo Finance Targets:")
        print("Recommendations:")
        print(yahoo_targets.get('recommendations', 'No recommendations available'))
        print("\nPrice Targets:")
        price_targets = yahoo_targets.get('price_targets', {})
        for key, value in price_targets.items():
            print(f"{key.replace('_', ' ').title()}: {value}")
    else:
        print("Failed to retrieve stock information.")

if __name__ == '__main__':
    main()

# Important Notes:
# 1. This script requires yfinance library
# 2. Install dependencies: pip install yfinance requests
# 3. Be aware of potential rate limits or changes in Yahoo Finance's structure

# Troubleshooting:
# - Ensure you have the latest version of yfinance
# - Some stock symbols might not have complete information
# - Network connectivity can affect data retrieval
   

Yahoo Finance Targets:
Recommendations:
  period  strongBuy  buy  hold  sell  strongSell
0     0m          2    3     0     0           0
1    -1m          2    3     0     0           0
2    -2m          2    3     0     0           0
3    -3m          2    3     0     0           0

Price Targets:
Current Price: 25.645
Target High Price: 53.0
Target Low Price: 15.0
Target Mean Price: 35.94
Target Median Price: 36.0


In [5]:
import smtplib
from email.mime.text import MIMEText

SMTP_SERVER = "smtp.zoho.com"
SMTP_PORT = 587
EMAIL_ADDRESS = "trendsense@zohomail.com"
EMAIL_PASSWORD = "pZNUVbUid0tv"
USER_EMAIL_ADDRESS = "4064315613@vtext.com"

try:
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        message_body = "Test email from Cloud Function."
        msg = MIMEText(message_body)
        msg['Subject'] = "Test Email"
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = USER_EMAIL_ADDRESS
        server.sendmail(EMAIL_ADDRESS, USER_EMAIL_ADDRESS, msg.as_string())
    print("Email sent successfully.")
except Exception as e:
    print(f"Error sending email: {e}")

Email sent successfully.


In [25]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from newspaper import Article
import nltk

# Ensure the required NLTK data is downloaded
nltk.download('punkt')

def fetch_article_summary(link):
    """
    Fetch and summarize the article content from a URL.
    """
    try:
        # Use Newspaper3k with headers
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"[ERROR] Newspaper3k failed for {link}: {str(e)}. Falling back to BeautifulSoup.")

        # Fallback to BeautifulSoup
        try:
            response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            content = " ".join([p.get_text() for p in paragraphs])
            return content[:500] + "..." if len(content) > 500 else content
        except Exception as bs_error:
            print(f"[ERROR] BeautifulSoup also failed for {link}: {str(bs_error)}")
            return "No summary available."




def get_market_news(tickers):
    """
    Fetch market news for the current day, capturing all available fields and generating summaries.
    Only processes news items with 'type' set to 'story'.
    """
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)

        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Filter news to include only today's and yesterday's articles
                    if publish_date >= one_day_ago:
                        # Only summarize articles with type 'story'
                        if item.get('type', '').lower() == 'story':
                            link = item.get('link', '')
                            summary = fetch_article_summary(link) if link else "No summary available."

                            news_item = {
                                'ticker': ticker,
                                'title': item.get('title', ''),
                                'publisher': item.get('publisher', ''),
                                'link': link,
                                'publish_date': datetime.fromtimestamp(publish_timestamp),
                                'summary': summary,  # Include the generated summary
                                'type': item.get('type', ''),  # Original type from Yahoo API
                                'related_tickers': ', '.join(item.get('relatedTickers', [])),  # Comma-separated related tickers
                            }
                            all_news.append(news_item)
                        else:
                            print(f"[INFO] Skipping non-story type: {item.get('type', '')}")
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")

        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")

    print(f"[INFO] Fetched {len(all_news)} news articles.")
    return pd.DataFrame(all_news)


def save_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file locally.
    """
    try:
        if df.empty:
            print("[INFO] No news data to save.")
            return None

        os.makedirs(output_dir, exist_ok=True)
        filename = f"market_news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False, encoding="utf-8")
        print(f"[INFO] News data saved locally at: {filepath}")
        return filepath
    except Exception as e:
        print(f"[ERROR] Failed to save CSV: {str(e)}")
        return None


def main():
    """
    Main function for fetching and saving market news locally.
    """
    try:
        # Fetch general market news
        indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
        market_news = get_market_news(tickers=indices)
        if not market_news.empty:
            market_news['category'] = 'General'  # Add category for general market

        # Fetch tech stock news
        tech_stocks = ['AAPL', 'GOOGL', 'MSFT']
        tech_news = get_market_news(tickers=tech_stocks)
        if not tech_news.empty:
            tech_news['category'] = 'Tech'  # Add category for tech stocks

        # Combine news
        combined_news = pd.concat([market_news, tech_news], ignore_index=True)

        # Save to CSV locally
        if not combined_news.empty:
            save_to_csv(combined_news)
        else:
            print("[INFO] No news data to save.")
    except Exception as e:
        print(f"[ERROR] Error in main function: {e}")


if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BryceDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[INFO] Skipping non-story type: VIDEO
[INFO] Skipping non-story type: VIDEO
[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/p-500-closes-record-high-213627406.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\OneDrive